# Kaggle Competition: Data Science Bowl 2017

<img src="assets/kaggle/logo.png" align="left" alt="Data Science Bowl 2017" style="width: 900px;"/>

### Our Team
<p align="center">
<img src="" align="left" style="width: 85px;"/><img src="assets/kaggle/chloe.jpeg" align="left" style="width: 85px;"/><img src="assets/kaggle/ricky.jpeg" align="left" style="width: 85px;"/><img src="assets/kaggle/martin.png" align="left" style="width: 85px;"/><img src="assets/kaggle/matt.jpeg" align="left" style="width: 85px;"/>
</p>

[Chloe Pont](https://www.linkedin.com/in/chloepont/) | 
[Ricky Lui](https://www.linkedin.com/in/mingluiricky/) | 
[Martin Simanjuntak](https://www.linkedin.com/in/martinsimanjuntak/) | 
[Matthew Ho](https://www.linkedin.com/in/matthewwilfred/)

We are a team of Data Scientists based out of General Assembly London, here to give lung cancer a good kicking!

### Project outline

This is our roadmap to facing this challenge:

- Data handling with Python, `numpy` and `pandas`
- Image preprocessing to deal with heterogeneous data using `SimpleITK`
- **Deep Learning with `Keras`, specifically using Convolutional Neural Networks for image recognition**
- Explore data augmentation and/or transfer learning to facilitate convergence with low generalisation error
- Amazon EC2 instances and S3 bins for training our Neural Networks
- Machine Learning models, in particular ensemble methods, will be trialled and evaluated on Validation set performance

# Project Milestone 2 - Testing 3D Convolutional Neural Networks

To recap the nature of our project: this is a __Supervised Machine Learning problem__ of __Binary Classification__. It is also an __image recognition problem__, one which we will apply Convolutional Neural Networks (ConvNets) on. A reminder of some key facts and figures of our dataset:

- 3D Lung scans of 1397 patients, consisting of 'stacks' of 2D images for each patient
- 26% of the entire Training/Validation set is labelled cancerous, with the remaining labelled non-cancerous

From our last project milestone, we have demonstrated the workflow for image preprocessing, and have transformed our set of 512x512x(variable) set of patient lung scan images into single-channel images with dimensions 64x64x64. That workflow has been condensed down into our `load_data` module, which we will import as part of our data ETL pipeline.

<img src="https://upload.wikimedia.org/wikipedia/commons/c/c9/Keras_Logo.jpg" align="right" style="width: 60px;"/>
We will also be importing Keras, a high level Deep Learning library that was created by François Chollet, an artificial intelligence researcher at Google. It is a popular library that is designed for fast prototyping, through its key features of modularity, minimalism, and extensibility. Using this module, we can rapidly construct the 3D convolutional networks for our image recognition problem.

<img src="https://upload.wikimedia.org/wikipedia/commons/a/a4/TensorFlowLogo.png" align="left" style="width: 100px;"/>
Though initially designed for a Theano backend, Keras also supports Tensorflow, which was again created by Google and open sourced in 2015. Since its release, it has enjoyed immense growth and popularity among Deep Learning researchers. One of its key strengths is an out-of-the-box capability to parallelise its computations across clusters, which is a boon for neural network training that remains to be the most significant time bottleneck for complicated Deep Learning applications.

We have chosen Tensorflow as the backend precisely for this reason. Because of the forseeably expensive computational resources required for training our 3D ConvNet, we plan to deploy our model on Amazon EC2 GPU enabled instances. GPUs have in recent years enjoyed an increase in usage because of Deep Learning, because its architecture allows rapid parallel computations that can much more efficiently solve linear algebra problems that would be much slower with CPUs.

This notebook details our first working 3D ConvNet, which we have trialled on the image data from one patient. Our network was able to train and converge using a MacBook Pro. This prototyping serves to demonstrate that we have resolved the issue of memory constraints through our image preprocessing, and we can proceed from here to refine our model, and train it on the entire corpus of our patient data.

## Importing libraries.

On top of our usual data handling libraries, we will also load in Keras and Tensorflow for our neural network, and SimpleITK along with our image preprocessing code and our patient's data:

In [1]:
import keras
import tensorflow
import numpy as np
import pandas as pd
import load_data # image preprocessing code from Project Milestone 1
import SimpleITK
np.random.seed(42) #Make sure results can be reproduced
from keras.models import Sequential #Feed forward
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Convolution3D, MaxPooling3D
from keras.utils import np_utils

labels = pd.read_csv('assets/kaggle/stage1_labels.csv') #load the CSV containing the labels
patient_id = '0a0c32c9e08cc2ea76a71649de56be6d' #Our selected patient ID
patient1 = load_data.get_lungs_arr(patient_id, 'patient_image/') #Use preprocessing code to obtain 64x64x64 3D image for our patient
# patient_image/ not included in Github due to file size

Using TensorFlow backend.


Here it must be noted that using Keras with the Tensorflow backend or Theano backend carries one important difference: __the expected dimensions of the input vectors are different.__ Reshaping to the appropriate dimension is required.

In [14]:
# Using TensorFlow, images are represented as NumPy arrays with the shape (height, width, depth), where the depth is the number of channels in the image.

# However, if you are using Theano, images are instead assumed to be represented as (depth, height, width).

X_train = patient1.reshape(1,64,64,64,1)

y_train = np_utils.to_categorical(labels[labels['id']==patient_id]['cancer'].values, 2)
# Change labels to array. E.g. instead of having 0 or 1 we have [1,0] or [0,1] where the index of '1' is 
# the true label

## Constructing the Convolutional Neural Network.

A Convolutional Neural Network is a Feedforward network, in contrast with other types such as Recurrent Neural Networks which incorporate feedback mechanisms in the network architecture. That being said, a lot of the recent advances in Deep Learning come from novel architectures that have elements of both, or can be completely different such as Generative Adversarial Networks. 

A ConvNet consists of a number of layers, where every layer transforms one volume of activations to another through an activation function. Stripped down to the bare minimum, there are three types of layers that are essential to building a ConvNet: 

- Convolutional Layer
- Pooling Layer
- Dense Layer aka Fully-Connected Layer 

Beyond these key layers, it is quite important also to have Dropout and Flatten implemented, we will see more of that below as we construct our ConvNet.

First instantiate our model using the `Sequential()` class implemented in Keras. The word sequential indicates that it is a 1 by 1 Feed forward network; naturally, other architectures such as RNNs would require a different class.

In [3]:
model = Sequential() 
# Feed forward 1 by 1 hence sequential. Other architectures such as LSTM would be different

Next, we add layers to the model. In this case our first layer is a 3D convolution layer. The first 4 arguments to `Convolution3D()` represents the **number of filters**, and then the dimensions in the order of **depth, width and height**. The difference between 3D ConvNets and 2D ConvNets is an extra dimension to the input arrays.

<img src="http://csci431.artifice.cc/images/relu.png" align="right" style="width: 300px;"/>
The activation function is applied to the sum of weighted inputs to generate the output to be passed onto the next hidden layer. Our chosen ReLU stands for Rectified Linear Unit, which is nothing more than this graphed out function to the right. 

On the topic of ReLU, the _Deep Learning_ book by Ian Goodfellow et al. states that the theory behind the choice of activation functions for hidden units is as yet not very well developed, and that barring extensive trial and error to prove otherwise, it can be a perfectly reasonable choice to choose ReLU. Although the differentiability of functions is essential to the functioning of back-propagation to train NNs, it is not necessary for such functions to be differentiable at all points; the ReLU for example is non-differentiable at the origin, where there is a discontinuity between the flat and sloping parts. 

The Rectified Linear Units use the activation function $R(z) = max(0,z)$. It is intuitive to understand: it takes negative inputs and turns them to zero, and keeps the positive inputs. It is attractive because its gradients are both large and consistent for back-propagation. This characteristic is also why neurons should be initialised with small positive bias values: this will give it an initial positive output, and hence a gradient for back-propagation to work on during training. This linear aspect of ReLU renders computations easier as opposed to non-linear activation functions such as Sigmoids.

Back to our code. Since this is the first layer of the model, we will need an input_shape parameter as well. The keyword argument `dim_ordering` is just the parameter telling the model that we will use TensorFlow convention for the dimesion ordering, as opposed to Theano convention.

In [15]:
model.add(Convolution3D(8,10,10,10, activation='relu',input_shape=(1,64,64,64),dim_ordering='tf'))
model.output_shape

(None, 46, 46, 46, 8)

<img src="assets/kaggle/Kaggle_DSB_network.png" align="center" style="width: 900px;"/>

Let us then walk through the successive steps we'll be taking to build our network. In this order we have:

- Convolution3D
- MaxPooling3D
- Dropout
- Convolution3D
- MaxPooling3D
- Dropout
- Flatten
- Dense (aka Fully Connected)
- Dropout
- Dense (aka Fully Connected)
- Softmax

**Convolution3D** layers take our resized single-channel input images of dimension 64x64x64, and applies 8 different filters of 3 by 3 matrices. As the ConvNet is trained, the weights associated with each layer are progressively adjusted during back-propagation to more heavily weight the important features, and vice versa. The 'Convolution' that gives ConvNets their name come from the action of convolving, where each 3x3 matrix slides over the input matrix. After a matrix multiplication with the associated weights. Our ReLU activation function is then applied elementwise to give the output of that layer.

**MaxPooling** performs a downsampling operation, where the dimensions of the input tensor are decreased to result in a smaller volume. This results in dimensionality reduction of the feature space thus helping with the computational complexity of our ConvNet, and also helps reduce overfitting. Concretely, a 2x2 matrix is convolved over the image from the preceding layer, and outputs the maximum value of the 2x2 area.

**Flatten** simply reshapes our image matrices into a 2D array for input into the next layer - visually, the volume disappears into a flat sheet, hence the name.

**Dropout** randomly selects a specified proportion (25% in our case) of units in the preceding layer, then sets their output values to zero. This has a regulsarisation effect, because all neurons must become more robust and generalisable to compensate for this random but temporary 'pruning' of the network during the training phase.

**Dense** or **Fully Connected** layers are characterised by each of its input neurons being connected to all its output neurons.

**Softmax** units are applied for Multinoulli output distributions i.e. n-class classification problems. Its name comes from it being a ’soft’, or continuous and differentiable version of the argmax (arguments of the maxima) function. It can be viewed as a generalisation of the sigmoid function over n different classes. For each class, the softmax produces a probability between 0 and 1, with the probability between all n classes summing to 1.

In [17]:
model.add(Convolution3D(8,10,10,10,activation='relu',dim_ordering='tf'))
print(model.output_shape)
model.add(MaxPooling3D(pool_size=(2,2,2))) # Reduce number of parameters
print(model.output_shape)
model.add(Convolution3D(8,5,5,5,activation='relu',dim_ordering='tf'))
print(model.output_shape)
model.add(MaxPooling3D(pool_size=(2,2,2))) # Reduce number of parameters
print(model.output_shape)
model.add(Dropout(0.25)) # Regularised the model to prevent overfit
print(model.output_shape)
# Now we have 2x conv + 2 maxpool + 1 dropout
model.add(Flatten()) # Must flattern to pass into dense
print(model.output_shape)
model.add(Dense(128, activation='relu')) # 
print(model.output_shape)
model.add(Dropout(0.5))
print(model.output_shape)
model.add(Dense(2))
model.add(Activation('softmax')) # 2 output matches binary classification criteria
print(model.output_shape)

(None, 37, 37, 37, 8)
(None, 18, 18, 18, 8)
(None, 14, 14, 14, 8)
(None, 7, 7, 7, 8)
(None, 7, 7, 7, 8)
(None, 2744)
(None, 128)
(None, 128)
(None, 2)


Having specified the architecture of our model, there remain three things we need to add to the model before we can train the network:

- **Cost function**: Categorical Cross Entropy
- **Optimiser**: Adam, [a method for stochastic optimisation](https://arxiv.org/abs/1412.6980v8).
- **Metric**: Accuracy



For more information on optimisation algorithms used in neural networks, [this blog post](http://sebastianruder.com/optimizing-gradient-descent/) by Sebastian Ruder gives an excellent overview of many different optimisers and their relative strengths and weaknesses. It belongs to the family of Gradient Descent optimisation algorithms, and has been shown empirically to work well in practice, which is why it was chosen. Intuitively, it makes use of the momentum, or moving averages of the parameters being optimised, to find an optimal learning step size.

In [18]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

Up to this point, we have finished specifying the architecture of our 3D ConvNet in Keras, which in Tensorflow means we have constructed the entire computation graph. It is now time to train the network, which Keras conveniently implements with the `fit()` method that it shares with `scikit-learn`:

In [19]:
model.fit(X_train,y_train)

Epoch 1/10
1/1 [==============================] - 418s - loss: 4.2841 - acc: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 437s - loss: 1.1921e-07 - acc: 1.0000
Epoch 3/10
1/1 [==============================] - 477s - loss: 1.1921e-07 - acc: 1.0000
Epoch 4/10
1/1 [==============================] - 418s - loss: 1.1921e-07 - acc: 1.0000
Epoch 5/10
1/1 [==============================] - 417s - loss: 1.1921e-07 - acc: 1.0000
Epoch 6/10
1/1 [==============================] - 415s - loss: 1.1921e-07 - acc: 1.0000
Epoch 7/10
1/1 [==============================] - 390s - loss: 1.1921e-07 - acc: 1.0000
Epoch 8/10
1/1 [==============================] - 373s - loss: 1.1921e-07 - acc: 1.0000
Epoch 9/10
1/1 [==============================] - 372s - loss: 1.1921e-07 - acc: 1.0000
Epoch 10/10
1/1 [==============================] - 372s - loss: 1.1921e-07 - acc: 1.0000


We have specified Accuracy to be our metric, which of course for one patient's data would result in massive overfitting! But that is not the point for this project, because this is what we have achieved: 

###  We now have our proof of concept. Our results show that our preprocessed images can be successfully trained on our 3D Convolutional Neural Network. We can proceed to iterate on and extend our ConvNet to make the best possible image classifier.

## Next Steps.

- **Train our ConvNet on the entire dataset.** To speed up the process, we will be using GPU enabled Amazon EC2 instances. The architecture and hyperparameters will be varied and evaluated for optimal performance. Techniques such as data augmentation or transfer learning will also be used as appropriate.
- **Model evaluation.** We have our usual arsenal of tools for classification models of Confusion Matrices, ROC curves, False Positive Rate etc.
- **Model visualisation.** We can look at the features that have been picked out by the different layers of our network, to try and tease out the key features that our network identifies.

Stay tuned for Parts 3 and more as we progress along!

<p align="center">
<img src="" align="left" style="width: 85px;"/><img src="assets/kaggle/chloe.jpeg" align="left" style="width: 85px;"/><img src="assets/kaggle/ricky.jpeg" align="left" style="width: 85px;"/><img src="assets/kaggle/martin.png" align="left" style="width: 85px;"/><img src="assets/kaggle/matt.jpeg" align="left" style="width: 85px;"/>
</p>

[Chloe Pont](https://www.linkedin.com/in/chloepont/) | 
[Ricky Lui](https://www.linkedin.com/in/mingluiricky/) | 
[Martin Simanjuntak](https://www.linkedin.com/in/martinsimanjuntak/) | 
[Matthew Ho](https://www.linkedin.com/in/matthewwilfred/)